## Deskewing and saving using simple for loops

In [3]:
import dask
import dask.array as da
from pyclesperanto_prototype import imshow

import pyclesperanto_prototype as cle

from aicsimageio import AICSImage, writers
from aicsimageio.types import PhysicalPixelSizes

from functools import partial
import numpy as np
import os 

from tqdm import tqdm

from skimage.io import imsave

cle.get_device()

C:\Users\Pradeep\.conda\envs\napari-lattice\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


<NVIDIA GeForce RTX 3080 on Platform: NVIDIA CUDA (1 refs)>

In [5]:
data_path = 'C:\\RAPA_treated-01_resaved_c02_t_100.czi'
data = AICSImage(data_path)

voxel_size_x_in_microns = data.physical_pixel_sizes.X
voxel_size_y_in_microns = data.physical_pixel_sizes.Y
voxel_size_z_in_microns = data.physical_pixel_sizes.Z

#eventually get angle from metadata
deskewing_angle_in_degrees = 30

Simple for loop to save the deskewed images as individual tiffs using scikit image imsave

In [6]:
save_path = "D:\\deskew_save\\"
save_name = "deskewed"

#create aicsimageio physical pixel size variable using PhysicalPixelSizes class to save pixel size
aics_image_pixel_sizes = PhysicalPixelSizes(data.physical_pixel_sizes.Z,data.physical_pixel_sizes.Y,data.physical_pixel_sizes.X)


#time_range = range(data.dims.T)
#we will only the first few timepoints
time_range = range(3)
channel_range = range(data.dims.C)


for time_point in tqdm(time_range, desc = "Time", position=0):
        
    for ch in tqdm(channel_range, desc = "Channels", position=1,leave = False): 
        image_to_deskew = data.get_image_dask_data("ZYX",T=time_point,C=ch,S=0) #scene is zero for now
        deskewed = cle.deskew_y(image_to_deskew, angle_in_degrees = deskewing_angle_in_degrees , 
                                voxel_size_x = voxel_size_x_in_microns, voxel_size_y= voxel_size_y_in_microns, voxel_size_z = voxel_size_z_in_microns)
        final_name=save_path+os.sep+"C"+str(ch)+"_T"+str(time_point)+"_"+save_name+".ome.tif"
        imsave(final_name, deskewed)

Time: 100%|██████████| 3/3 [00:13<00:00,  4.61s/it]


Using Ometiffwriter gives me this error:

In [7]:
save_path = "D:\\deskew_save\\"
save_name = "deskewed"

#create aicsimageio physical pixel size variable using PhysicalPixelSizes class to save pixel size
aics_image_pixel_sizes = PhysicalPixelSizes(data.physical_pixel_sizes.Z,data.physical_pixel_sizes.Y,data.physical_pixel_sizes.X)


#time_range = range(data.dims.T)
#we will only the first few timepoints
time_range = range(3)
channel_range = range(data.dims.C)


for time_point in tqdm(time_range, desc = "Time", position=0):
        
    for ch in tqdm(channel_range, desc = "Channels", position=1,leave = False): 
        image_to_deskew = data.get_image_dask_data("ZYX",T=time_point,C=ch,S=0) #scene is zero for now
        deskewed = cle.deskew_y(image_to_deskew, angle_in_degrees = deskewing_angle_in_degrees , 
                                voxel_size_x = voxel_size_x_in_microns, voxel_size_y= voxel_size_y_in_microns, voxel_size_z = voxel_size_z_in_microns)
        final_name=save_path+os.sep+"C"+str(ch)+"_T"+str(time_point)+"_"+save_name+".ome.tif"
        writers.OmeTiffWriter.save(deskewed, final_name, physical_pixel_sizes = aics_image_pixel_sizes)

Time:   0%|          | 0/3 [00:01<?, ?it/s]


AttributeError: 'OCLArray' object has no attribute 'itemsize'